# ISE Cryptography Week 3

This week's challenges are mostly centred around being extremely cruel to various kinds of block cipher modes of operation!

## Cryptography in Python

Most of these exercises are going to feel a lot more convincing if you try them against a state-of-the-art block cipher like AES.
I recommend using the [`pycryptodome`](https://pycryptodome.readthedocs.io/en/latest/src/introduction.html) library for this, which is a fork of the older `pycrypto` library.

```python
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_ECB)

plaintext = b"Hello, world!"
ciphertext = cipher.encrypt(plaintext)
decrypted = cipher.decrypt(ciphertext)

print(decrypted)
```

In [7]:
# Install pycryptodome (useful if you're running this in Google Colab)
%pip install pycryptodome

import secrets
from Crypto.Cipher import AES

## Cracking ECB Mode with a Chosen Plaintext Attack

By just being able to control the prefix of the plaintext, we can completely break ECB mode, no matter how strong the block cipher is!

Demonstrate this by writing a function to crack ECB mode, given a function that encrypts arbitrary plaintexts and appends a secret string to them.

You should print the secret string at the end.

In [47]:
def aes_ecb_encrypt_with_secret(plaintext: bytes):
    # Add a secret message to the end of the plaintext
    plaintext += " This is a very original secret message!".encode("utf-8")
    # Pad the plaintext to a multiple of 16 bytes
    plaintext += b"\x00" * (16 - len(plaintext) % 16)
    # Please don't hardcode the key in a real-world scenario!
    key = b"\xa1" * 16
    # Encrypt the plaintext with AES-128-ECB
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

plaintext = b"Hello, World!"
ciphertext = aes_ecb_encrypt_with_secret(plaintext)

def crack_ecb_mode(encrypt_func):
    block_size = AES.block_size
    known_bytes = b''
    num_blocks = len(encrypt_func(b'')) // block_size

    for _ in range(num_blocks):
        for i in range(block_size):
            dictionary = {}
            for j in range(256):
                payload = b'\x00' * (block_size - 1 - i) + known_bytes + bytes([j])
                ciphertext_block = encrypt_func(payload)[:block_size]
                dictionary[ciphertext_block] = bytes([j])
            ciphertext_block = encrypt_func(b'\x00' * (block_size - 1 - i))[:block_size]
            known_bytes += dictionary[ciphertext_block]

    return known_bytes

plaintext = b"Hello, World!"
ciphertext = aes_ecb_encrypt_with_secret(plaintext)

print("Secret Message:", crack_ecb_mode(aes_ecb_encrypt_with_secret))


Secret Message: b' This is a very \xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'


## Attacking CBC Mode with a Constant IV

Implement the attack discussed in the lecture slides against CBC mode with a predictable IV. In this case, the IV is always set to zero.
Demonstrate that this allows you to verify the correctness of a guess for the plaintext under a chosen-plaintext attack.

In [9]:
def cbc_constant_iv_encrypt(plaintext: bytes):
    # Pad the plaintext to a multiple of 16 bytes
    plaintext += b"\x00" * (16 - len(plaintext) % 16)
    # Encrypt the plaintext with AES-128-CBC
    key = b"\xf5" * 16
    iv = b'\x00' * 16
    cipher = AES.new(key, AES.MODE_CBC, iv)
    # Note that we're returning the IV as well!
    return iv + cipher.encrypt(plaintext)

def cbc_constant_iv_attack():
    # Let's encrypt a test score!
    ciphertext = cbc_constant_iv_encrypt(b"42%")
    # TODO: Implement the attack here! You can call cbc_constant_iv_encrypt() as many times as you want.
    # TODO: Show that, because you know the IV that will be used, you can verify if a guessed plaintext is correct.
    # TODO: For this exercise, you can assume that the plaintext is a percentage score between 0% and 100% to make brute force guessing easier.
    # TODO: The function should return the test score that was encrypted.

## Attacking CBC Mode with the Key as IV

Implement the attack discussed in the lecture slides against CBC mode with the key as the IV.
Demonstrate that this allows you to recover the key under a chosen-ciphertext attack,
i.e. you can encrppt arbitrary plaintexts and decrypt arbitrary ciphertexts.

Note that we're using AES-128 here, so the key is 16 bytes long - the same size as a block.
If you're using a different block cipher, the key might not fit into a single block!

In [10]:
def cbc_key_as_iv_encrypt(plaintext: bytes):
    # Pad the plaintext to a multiple of 16 bytes
    plaintext += b"\x00" * (16 - len(plaintext) % 16)
    # Encrypt the plaintext with AES-128-CBC
    key = b"\x42" * 16
    cipher = AES.new(key, AES.MODE_CBC, key)
    # Note that we're NOT returning the IV, since it's the same as the key!
    return cipher.encrypt(plaintext)


def cbc_key_as_iv_decrypt(plaintext: bytes):
    # Pad the plaintext to a multiple of 16 bytes
    plaintext += b"\x00" * (16 - len(plaintext) % 16)
    # Encrypt the plaintext with AES-128-CBC
    key = b"\x42" * 16
    cipher = AES.new(key, AES.MODE_CBC, key)
    return cipher.encrypt(plaintext)


def cbc_key_as_iv_attack():
    # TODO: Use the encrypt and decrypt functions to recover the key used for encryption, i.e. the IV.
    pass

## Padding

Implement a zero-padding function and a function to remove zero-padding.
You can optionally allow the user to specify the byte that should be used for padding, but default to `\x00`.

Once you've done that, implement a padding and unpadding function for PKCS#7 padding. Don't forget to write some test cases!

Optionally, do a little research and implement another padding scheme of your choice.

In [11]:
class PaddingError(Exception):
    pass

def zero_pad(plaintext: bytes, block_size: int, padding_byte: bytes = b"\x00"):
    pass


def zero_unpad(padded_plaintext: bytes, padding_byte: bytes = b"\x00"):
    pass


def pkcs7_pad(plaintext: bytes, block_size: int):
    pass


def pkcs7_unpad(padded_plaintext: bytes):
    # TODO: Before unpadding, check if the padding is correct. If it's not, raise a PaddingError.
    pass

## CBC Padding Oracle Attack

Implement the padding oracle attack against CBC mode as discussed in the lecture slides.

In [12]:
def cbc_padding_oracle_encrypt(plaintext: bytes):
    # Pad the plaintext to a multiple of 16 bytes
    plaintext = pkcs7_pad(plaintext, 16)
    key = b"\x83" * 16
    iv = secrets.token_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(plaintext)
    return iv + ciphertext


def cbc_padding_oracle(ciphertext: bytes) -> bool:
    # Extract the IV and the ciphertext
    iv, ciphertext = ciphertext[:16], ciphertext[16:]
    # Decrypt the ciphertext
    key = b"\x83" * 16
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)
    try:
        # Unpad the plaintext
        pkcs7_unpad(plaintext)
        return True
    except PaddingError:
        return False


def cbc_padding_oracle_attack():
    # Keeping the plaintext shorter than 16 bytes to make the attack easier
    # Remember that the IV is prepended to the ciphertext!
    ciphertext = cbc_padding_oracle_encrypt(b"Hello, world!")
    # TODO: Implement the padding oracle attack here!
    # TODO: The function should return the plaintext that was encrypted.
    pass

## CBC-R Padding Oracle Attack

Now for a challenge: implement the padding oracle attack against CBC-R mode as mentioned in the lecture slides.
You'll need to start by doing some research into what this attack is and how it works.

After that, you'll need to set up a scenario and implement the attack itself!

In [13]:
# TODO: This time, it's all up to you!